# Acquire Exercises

The end result of this exercise should be a file named acquire.py.

In [1]:
import pandas as pd

import requests

## 1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named `items` that has all of the data for items.

In [5]:
domain = 'https://python.zgulde.net'
endpoint = '/api/v1/items'
items = []

url = domain + endpoint

response = requests.get(url)
data = response.json()
# .extend adds elemnts from a list to another list
items.extend(data['payload']['items'])

In [6]:
data['payload']['next_page']

'/api/v1/items?page=2'

In [7]:
url = domain + data['payload']['next_page']
print('Next url:', url)

Next url: https://python.zgulde.net/api/v1/items?page=2


In [8]:
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])

In [9]:
url = domain + data['payload']['next_page']
print('next url:', url)

next url: https://python.zgulde.net/api/v1/items?page=3


In [10]:
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])

In [11]:
# Hint hint: if data['payload']['next_page'] is None:
print('next endpoint', data['payload']['next_page'])

next endpoint None


In [104]:
itemsdf = pd.DataFrame(items)

## 2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [28]:
domain = 'https://python.zgulde.net'
endpoint = '/api/v1/stores'
stores = []

url = domain + endpoint

response = requests.get(url)
data = response.json()
# .extend adds elemnts from a list to another list
stores.extend(data['payload']['stores'])

In [31]:
print('next endpoint', data['payload']['next_page'])

next endpoint None


In [105]:
storesdf = pd.DataFrame(stores)

## 3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [91]:
domain = 'https://python.zgulde.net'
endpoint = '/api/v1/sales'
sales = []
url = domain + endpoint

page = 1
while page < data['payload']['max_page']:
    
    response = requests.get(url)
    data = response.json()
    # .extend adds elemnts from a list to another list
    sales.extend(data['payload']['sales'])
    # Set the url for the page
    url = domain + data['payload']['next_page']

    page +=1

# url = domain + data['payload']['next_page']
# response = requests.get(url)
# data = response.json()
# sales.extend(data['payload']['sales'])

In [98]:
url = domain + data['payload']['next_page']
response = requests.get(url)
data = response.json()
sales.extend(data['payload']['sales'])

In [99]:
salesdf = pd.DataFrame(sales)

In [100]:
salesdf.shape

(913000, 5)

In [101]:
salesdf.sale_id

0              1
1              2
2              3
3              4
4              5
           ...  
912995    912996
912996    912997
912997    912998
912998    912999
912999    913000
Name: sale_id, Length: 913000, dtype: int64

## 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [106]:
itemsdf.to_csv('items.csv', index =False)
storesdf.to_csv('stores.csv', index =False)
salesdf.to_csv('sales.csv', index =False)

## 5. Combine the data from your three separate dataframes into one large dataframe.

In [107]:
salesdf.columns

Index(['item', 'sale_amount', 'sale_date', 'sale_id', 'store'], dtype='object')

In [108]:
itemsdf.columns

Index(['item_brand', 'item_id', 'item_name', 'item_price', 'item_upc12',
       'item_upc14'],
      dtype='object')

In [110]:
storesdf.columns

Index(['store_address', 'store_city', 'store_id', 'store_state',
       'store_zipcode'],
      dtype='object')

In [111]:
salesdf = salesdf.rename(columns = {'item': 'item_id', 'store':'store_id'})

In [112]:
df = pd.merge(salesdf, itemsdf, how='left', on='item_id')
df = pd.merge(df, storesdf, how='left', on='store_id')

In [113]:
df.shape

(913000, 14)

## 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here:
https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [114]:
df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

In [115]:
df.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [116]:
def get_opsd():
    if os.path.exists('opsd.csv'):
        return pd.read_csv('opsd.csv')
    df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
    df.to_csv('opsd.csv', index=False)
    return df

## 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.